# Постановка задачи

SARS-Cov2 - РНК-вирус. Его геном состоит из одиночной РНК, в которой закодировано несколько генов, кодирующих белки, без некодирующих элементов. Последовательность, которую выскачаете - это ДНК последовательность для этой РНК (то есть РНК транскрибировали обратно чтобы получить ДНК). Последовательность находится тут.

Задача - проанализировать последовательность с помощью библиотеки biopython. Что, например, можно узнать про последовательность (это только примеры, вы можете придумать другие параметры, которые можно посмотреть для геномной последовательности):
- частоты встречаемости нуклеотидов A, G, C, T
- GC состав
- превратить нуклеотидную последовательность в РНК и в белки (сделать in silico транскрипцию и трансляцию)
- посчитать частоты кодонов
- чтобы узнать не только последовательности белков, но еще и их функции и что-либо про них - использовать blast
- поискать специальные библиотеки для визуализации геномных данных и красиво визуализировтаь ваши результаты (например, показать открытые рамки считывания)

В результате - рассказать какие функции и как вы использовали, какие результаты получили - визуализации и представление необходимых данных на ваш выбор


# Визуализация всей последовательности через `FluentDNA`


FluentDNA - https://github.com/josiahseaman/FluentDNA



Легенда:
![Легенда](assets/fluent_dna_legend.png)


ДНК:
![ДНК](assets/fluent_dna_visualization.png)



## Чтение последовательности последовательности

In [30]:
import plotly.express as px
import pandas as pd


from Bio import SeqIO

record = next(SeqIO.parse('covid_sequence.fasta', "fasta"))

seq = record.seq

## Частоты встречаемости нуклеотидов A, G, C, T

Использовали `seq.count` - функция позволяет подсчитать количество нуклеотида в цепочке.

In [31]:
from Bio.Seq import Seq

def get_nucleotides_frequency(seq: Seq) -> pd.DataFrame:
    total_count = len(seq)
    nucliotides = ('A', 'G', 'C', 'T')
    frequencies = []
    
    for nucliotide in nucliotides:
        frequencies.append(
            (nucliotide, seq.count(nucliotide) / total_count)
        )
    
    return pd.DataFrame(frequencies, columns=['nucliotide', 'frequency'])


frequency = get_nucleotides_frequency(seq)

fig = px.pie(frequency, values='frequency', names='nucliotide', title='Nucliotide frequencies')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

# GC состав

In [32]:
from Bio.SeqUtils import gc_fraction


value = gc_fraction(seq)
print(value)

0.3797277865097147


# Превратить нуклеотидную последовательность в РНК и в белки

In [33]:
def get_rna_seq(dna_seq: Seq) -> Seq:
    return dna_seq.transcribe()


def get_protein_seq(rna_seq: Seq) -> Seq:
    return rna_seq.translate()


rna_seq = get_rna_seq(seq)
print("RNA seq: ", rna_seq)
print("Protein seq: ", get_protein_seq(rna_seq))


proteins = get_protein_seq(rna_seq)
proteins = list(filter(lambda x: len(x) > 50, proteins.split('*')))
print(len(proteins))

RNA seq:  ...
Protein seq:  IKGLYLPR*QTNQLSISCRSVL*TNFKICVAVTRLHA*CTHAV*LITNYCR*QDTSNSSIFCRLLTVSSVLQPIISTSRFRPGVTER*DGEPCPWFQRENTRPTQFACFTGSRRARTWLWRLRGGGLIRGTSTS*RWHLWLSRS*KRRFAST*TALCVHQTFGCSNCTSWSCYG*AGSRTRRHSVRS*W*DTWCPCPSCGRNTSGLPQGSSS*ER**RSWWP*LRRRSKVI*LRRRAWH*SL*RFSRKLEH*T*QWCYP*THA*A*RRGIHSLCR*QLLWP*WLPS*VH*RPSSTCW*SFMHFVRTTGLY*H*EGCILLP*T*A*NCLVHGTF*KEL*IADTF*N*IGKEI*HLQWGMSKFCISLKFHNQDYSTKG*KEKA*WLYG*NSICLSSCVTK*MQPNVPFNSHEV*SLW*NFMADGRFC*SHLRILWH*EFD*RRCHYLWLLTPKCCC*NLLSSMSQFRSRT*A*SCRIP**IWLENHSS*GWSHYCLWRLCVLLCWLP*QVCLLGSTC*R*HRL*PYRCCWRRFRRS**QPS*NTPKRESQHQYCW*L*T**RDRHYFGIFFCFHKCFCGNCERFGL*SIQTNC*ILW*F*SYKRKS*KRCLEYW*TEINTESSLCICIRGCSCCTINFLPHS*NCSKFCACFTEGRYNNTRWNFTVFTETH*CYDVHI*FGY*QSSCNGLHYRWCCSVDFAVAN*HLWHCL*KTQTRP*LA*REV*GRCRVS*RRLGNC*IYLNLCL*NCRWTNCHLCKGN*GECSDIL*ACK*IFGFVC*LYHYWWS*T*SLEFR*NICHALKGIVQKVC*IQRRNWPTHASKSPKRNYLLRGRNTSHRSVNRGSCLENW*FTTIRTTY**SC*SSIGWYTSLY*RAYVARNQRHRKVLCPCT*YDGNKQYLHTQRRCTNKGYFW**HCDRSARLQECEYHF*T**KD**ST**EVLCLYS*TRYRSK*VRLCCGRCCHKNFA

/home/denis/Work/bioPython-project/venv/lib/python3.10/site-packages/Bio/Seq.py:2804: BiopythonWarning:

Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.



# Посчитать частоты кодонов

In [35]:
def get_codon_frequency(dna_seq: Seq) -> dict[str, int]:
    codon_freq = dict()
    for i in range(0, len(dna_seq), 3):
        codon = dna_seq[i:i+3]
        if codon not in codon_freq:
            codon_freq[str(codon)] = 0
        codon_freq[str(codon)] += 1

    total_count = len(dna_seq) // 3
    for codon, count in codon_freq.items():
        codon_freq[str(codon)] = count / total_count
    
    as_df = pd.DataFrame(list(codon_freq.items()), columns=['codon', 'frequency'])
    return as_df

frequency = get_codon_frequency(seq)
fig = px.pie(frequency, values='frequency', names='codon', title='Codon frequencies')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

# Открытые рамки считывания

Найти ORF получилось через [orffinder](https://www.ncbi.nlm.nih.gov/orffinder/). Там же есть и посмотрели визуализацию.

![ORF Visualization](assets/orf_visualization_demo.png)